In [1]:
from bqplot import pyplot as plt
import ipywidgets as widgets
import numpy as np
import paramiko
import ftplib
import sqlite3
from pathlib import Path
import pandas as pd


In [2]:
# generate some fake 
n = 2000
x = np.linspace(0.0, 10.0, n)
np.random.seed(0)
y = np.cumsum(np.random.randn(n)*10).astype(int)

In [3]:
fig_hist = plt.figure( title='Histogram')
hist = plt.hist(y, bins=25)

In [4]:
hist.bins = 10;

In [5]:
slider = widgets.IntSlider(description='Bins number', min=1, max=100, v_model=30)

In [6]:
widgets.link((hist, 'bins'), (slider, 'value'))

fig_lines = plt.figure( title='Line Chart')
lines = plt.plot(x, y)

fig_lines.layout.width = 'auto'
fig_lines.layout.height = 'auto'
fig_hist.layout.width = 'auto'
fig_hist.layout.height = 'auto'

grid_layout = widgets.GridspecLayout(5, 3)

grid_layout[:2, :] = fig_lines
grid_layout[2:4, :] = fig_hist
grid_layout[4, 1] = slider

grid_layout.layout.height = '1000px'

grid_layout

GridspecLayout(children=(Figure(axes=[Axis(scale=LinearScale()), Axis(orientation='vertical', scale=LinearScal…

In [7]:
selector = plt.brush_int_selector()
def update_range(*ignore):
    if selector.selected is not None and len(selector.selected) == 2:
        xmin, xmax = selector.selected
        mask = (x > xmin) & (x < xmax)
        hist.sample = y[mask]
selector.observe(update_range, 'selected')        

In [8]:
from IPython.display import IFrame
IFrame('https://sustainablecharging.nl/', width=1200, height=500)

In [9]:
from pyowm.owm import OWM
from pyowm.utils import timestamps
from datetime import datetime, timedelta, timezone

In [10]:
def get_weather():
    
    owm = OWM('1a4df9d4817c3d16e92b272d59531753')
    mgr = owm.weather_manager()
    one_call = mgr.one_call(lat=51.98787601885725, lon=5.950209138832937)
    #one_call.forecast_daily[0].temperature('celsius').get('feels_like_morn', None) #Ex.: 7.7
    forecast_hourly = one_call.forecast_hourly
    
    nr_of_weathers = len(forecast_hourly)
    Cloud = np.zeros(len(forecast_hourly))
    Temperature = np.zeros(len(forecast_hourly))
    #Wind_speed = np.zeros(len(forecast_hourly))
    #Wind_direction = np.zeros(len(forecast_hourly))
    Humidity = np.zeros(len(forecast_hourly))
    Air_pressure = np.zeros(len(forecast_hourly))

    for i in range(nr_of_weathers):
        dt = (datetime.fromtimestamp(forecast_hourly[i].ref_time)).strftime('%Y-%m-%d %H:%M:%S')
        Cloud[i] = forecast_hourly[i].clouds
        Temperature[i] = forecast_hourly[i].temperature('celsius')['temp']
        #Wind_speed[i] = forecast_hourly[i].wind()['speed']
        #Wind_direction[i] = forecast_hourly[i].wind()['deg']
        Humidity[i] = forecast_hourly[i].humidity
        Air_pressure[i] = forecast_hourly[i].pressure['press']*0.1
        #print(dt, forecast_hourly[i].temperature('celsius'),forecast_hourly[i].clouds )
    return Temperature, Air_pressure , Humidity, Cloud

In [11]:
#Temperature, Air_pressure , Humidity, Cloud = get_weather()

In [21]:
#print('Temperature',Temperature)
#print('Cloud', Cloud)
#print('Humidity', Humidity)
#print('Air_pressure', Air_pressure)

In [22]:
def get_modbus():
    
    host = "80.113.19.27"
    port = 22
    password = "controlsystem"
    username = "pi"

    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(host, port, username, password)

    ftp = ssh.open_sftp()
    
    data_d = ftp.chdir('/mnt/dav/Data')
    cwd=ftp.getcwd()
    path = Path.cwd()
    
    ftp.get("modbusData.db","modbusData.db",callback=None)
    
    conn = sqlite3.connect("modbusData.db")
    query = '''SELECT * from PV'''
    PV_data= pd.read_sql_query(query,conn)
    PV_data['Time']=pd.to_datetime(PV_data['Time'],unit='s')
    
    # PV data 
    PV_data= PV_data.sort_values(by='Time', ascending=True)
    PV_data['Time']=pd.to_datetime(PV_data['Time'],unit='s')
    PV_data = PV_data.set_index('Time')
    PV_data=PV_data.resample('60min').mean()
    PV_data = PV_data.tail(24)
    
    return PV_data

In [23]:
PV_data =get_modbus()

In [24]:
PV_data.head()

,No,V1,V2,V3,I1,I2,I3,P1,P2,P3,Q1,Q2,Q3,S1,S2,S3,PF1,PF2,PF3,F
Time,,,,,,,,,,,,,,,,,,,,
2021-05-19 10:00:00,26828.5,230.413889,229.430556,229.833333,21.701667,21.749167,21.713333,4990.000000,4980.000000,4980.833333,-28.333333,-34.166667,-19.166667,4585.000000,4566.666667,4572.500000,0.990778,0.990083,0.990972,50.008889
2021-05-19 11:00:00,26861.0,228.341379,227.465517,227.996552,1.359310,1.377931,1.396552,249.310345,245.172414,252.413793,-31.034483,-43.448276,-36.206897,294.827586,300.000000,304.137931,0.794034,0.771483,0.789517,50.000000
2021-05-19 12:00:00,26891.0,230.409677,229.477419,230.019355,7.256129,7.275484,7.288065,1666.451613,1659.677419,1671.290323,-50.322581,-58.064516,-39.677419,1639.354839,1638.387097,1644.193548,0.989613,0.988645,0.989387,50.006774
2021-05-19 13:00:00,26923.0,229.154545,228.221212,228.800000,19.075455,19.089091,19.096364,4351.818182,4335.454545,4348.181818,-20.909091,-29.090909,-25.454545,4314.545455,4303.636364,4309.090909,0.998818,0.998788,0.998848,50.000000
2021-05-19 14:00:00,26956.5,230.394118,229.517647,229.858824,22.330588,22.322647,22.320882,5458.235294,5438.823529,5446.764706,-21.176471,-26.470588,-22.058824,5094.705882,5079.705882,5081.470588,0.998176,0.998059,0.998088,50.000000
